In [ ]:
# for python?
!python3 -m pip install --upgrade pip

In [ ]:
!pip install fsspec==2025.3.2
!pip install gcsfs==2025.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [ ]:
# for dependencies ????
!pip install accelerate -qU

In [ ]:
# perameter efficient fine tuning technique
!pip install peft -qU

In [ ]:
#To create data set as an input
!pip install datasets -qU

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
#Transformer to Load the model
!pip install transformers[torch] -qU

In [ ]:

# compress and qunatize
!pip install bitsandbytes -qU

In [ ]:
import os
import random
import string
from datetime import datetime
from typing import List
import torch
import transformers
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

In [ ]:
!pip install --upgrade transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
#import model from hugging face for testing. This is a smallest possible model found
model_checkpoint = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
# Run an example prompt prediction/infrerence
prompt = "def hello_world():"
completion = model.generate(**tokenizer(prompt, return_tensors="pt").to("cuda"))
print(tokenizer.decode(completion[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():
    print("Hello World")

hello_world()

# 파


In [ ]:
#Prepare model for training
#memory optimization technique that trades computation for memory
GRADIENT_CHECKPOINTING = False
#specifies the number of training samples processed per device (e.g., GPU) in a single forward/backward pass. A batch size of 1 means the model processes one sample at a time per device
PER_DEVICE_TRAIN_BATCH_SIZE = 1
#Warmup steps define the number of initial training steps where the learning rate gradually increases from a small value (e.g., 0) to the target learning rate. Setting WARMUP_STEPS = 0 means no warmup is applied
WARMUP_STEPS = 0
# EPOCHS is number of iterations
EPOCHS = 100
#The learning rate controls the step size for updating model weights during optimization
LEARNING_RATE = 1e-5
#R is the rank of the low-rank matrices used to adapt the pre-trained model. A rank of 32 determines the dimensionality of the update matrices, balancing expressiveness and efficiency.
R = 32
#LORA_ALPHA is a scaling factor for LoRA updates, controlling the magnitude of the low-rank adaptation.
LORA_ALPHA = R
#LoRA dropout is the dropout rate applied to the low-rank adaptation layers during training. A value of 0.1 (10%) means that 10% of the LoRA weights are randomly set to zero during each training step, acting as a regularization technique to prevent overfitting and improve generalization.
LORA_DROPOUT = 0.1

In [ ]:
# Identify the layers of neural networks to specify which layers to adapt
def find_target_modules(model) -> List[str]:
    """
    Identify linear layers in the model and return as list.
    """
    layers = set()
    for name, module in model.named_modules():
        if "Linear" in str(type(module)):
            layer_type = name.split('.')[-1]
            layers.add(layer_type)

    return list(layers)

target_modules = find_target_modules(model)
target_modules

['fc_out', 'out_proj', 'lm_head', 'qkv_proj', 'fc_in']

In [ ]:
peft_config = LoraConfig(
    task_type="CAUSAL_LM", inference_mode=False, r=R, lora_alpha=LORA_ALPHA, lora_dropout=0.1, target_modules=target_modules
)

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
letters = string.ascii_letters

prompts = []
length = 100
for i in range(3):
    random_string = ''.join(random.choice(letters) for i in range(length))
    #prompts.append("def hello_world():" + random_string)
    prompts.append("def hello_world():" + "Hello GenAI Course"+str(i))

data = [{"text": x} for x in prompts]
data

[{'text': 'def hello_world():Hello GenAI Course0'},
 {'text': 'def hello_world():Hello GenAI Course1'},
 {'text': 'def hello_world():Hello GenAI Course2'}]

In [ ]:
dataset = Dataset.from_dict({"text": [item["text"] for item in data[:]]})
dataset['text']

['def hello_world():Hello GenAI Course0',
 'def hello_world():Hello GenAI Course1',
 'def hello_world():Hello GenAI Course2']

In [ ]:
#Tokenization converts raw text into a format the model can understand (numerical IDs).

tokenized_dataset = dataset.map(lambda x : tokenizer(x["text"]), batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    logging_steps=100,
    save_total_limit=1,
    output_dir=os.path.join('.', datetime.now().strftime("%Y%m%d%H%M%S")),
    gradient_checkpointing=GRADIENT_CHECKPOINTING
)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=train_args,
    callbacks=[],
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer, mlm=False
    )
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
#The Hugging Face Trainer class supports automatic integration with W&B for logging metrics like loss, accuracy, or evaluation scores


loss = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kkolah (kkolah-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,3.489400
200,0.133500
300,0.118800


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [ ]:
# Inference after fine-tuning
prompt = "def hello_world():"

completion = model.generate(**tokenizer(prompt, return_tensors="pt").to("cuda"), max_length=20)

print(tokenizer.decode(completion[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def hello_world():Hello GenAI Course1 Course2 Course3 Course4 Course5 Course6
